# 2024 CITS4012 Project

# Readme

notes for marker

# 1. Dataset Processing

## Import the libraries

In [102]:
import re
import json
import nltk
import pandas as pd

## Loading the datasets

In [103]:
# Load the training data
with open('train.json', 'r') as file:
    data = json.load(file)
    train_data = pd.DataFrame(data['data'], columns=data['columns'])

# Load the test data
with open('test.json', 'r') as file:
    data = json.load(file)
    test_data = pd.DataFrame(data['data'], columns=data['columns'])
    
# Load the validation data
with open('val.json', 'r') as file:
    data = json.load(file)
    val_data = pd.DataFrame(data['data'], columns=data['columns'])

print("Training data size\t", train_data.shape)
print("Test data size\t\t", test_data.shape)
print("Validation data size\t", val_data.shape)


Training data size	 (7090, 3)
Test data size		 (901, 3)
Validation data size	 (888, 3)


In [104]:
# Print to visualise data
for i in range(0, 6, 2):
    print(train_data['sentence'][i])
    print(train_data['aspect'][i], "=", train_data['polarity'][i])
    print(train_data['aspect'][i+1], "=", train_data['polarity'][i+1])

It might be the best sit down food I've had in the area, so if you are going to the upright citizen brigade, or the garden, it could be just the place for you.
food = positive
place = neutral
Hostess was extremely accommodating when we arrived an hour early for our reservation.
staff = positive
miscellaneous = neutral
We were a couple of minutes late for our reservation and minus one guest, but we didn't think we deserved the attitude we got from the hostess.
miscellaneous = neutral
staff = negative


## Data Preprocessing

In [ ]:
# Punctuation Removal
# maybe keep emoticons !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
# handle contractions (i've -> i have)
def remove_punctuation_re(x):
    x = re.sub(r'[^\w\s]','',x)
    return x

nltk.download('punkt')

# Stopwords Removal
nltk.download('stopwords')
from nltk.corpus import stopwords as sw
from nltk.tokenize import word_tokenize
stopwords = sw.words('english')

# Stemming
from nltk.stem import PorterStemmer
stemmer = PorterStemmer()

# Lemmatisation
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

# POS Tagging
nltk.download('averaged_perceptron_tagger')
from nltk.tag import pos_tag

In [113]:
for i in range(len(train_data)):
    
    sentence = train_data.loc[i, 'sentence']
    
    # Lowercase
    sentence = sentence.lower()
    # print("1", sentence)
    
    # Tokenise
    tokens = word_tokenize(sentence)
    # print("2", tokens)
    
    # Remove punctuation
    re_tokens = [remove_punctuation_re(word) for word in tokens]
    # print("3", re_tokens)
    
    # Remove stopwords
    sw_tokens = [word for word in re_tokens if word.lower() not in stopwords and word != '']
    # print("4", sw_tokens)
    
    # Stemming
    stem_tokens = [stemmer.stem(word) for word in sw_tokens]
    # print("5", stem_tokens)
    
    # Lemmatisation
    lemma_tokens = [lemmatizer.lemmatize(word) for word in stem_tokens]
    # print("6", lemma_tokens)
    
    # POS Tagging
    pos_tokens = pos_tag(lemma_tokens)
    # print("7", pos_tokens)
    
    # Reconstruct sentence
    sentence = " ".join(lemma_tokens)
    

# for i in range(len(test_data)):
#     test_data.loc[i, 'sentence'] = remove_punctuation_re(test_data.loc[i, 'sentence'])

# for i in range(len(val_data)):
#     val_data.loc[i, 'sentence'] = remove_punctuation_re(val_data.loc[i, 'sentence'])


# 2. Model Implementation

## Model 1

## Model 2

## Model 3

# 3. Testing and Evaluation